In [1]:
#importing dependecies
#import praw  #not being used
import pandas as pd
#import config #not being used
from psaw import PushshiftAPI 
import datetime as dt

In [2]:
api = PushshiftAPI()

In [12]:
#adapted from https://psaw.readthedocs.io/en/latest/

gen = api.search_comments(q='GME', subreddit='wallstreetbets',filter=['body'],before=int(dt.datetime(2021, 1, 30,18,1).timestamp()))

max_response_cache = 10000
cache = []

for c in gen:
    cache.append(c[-1])

    # Omit this test to actually return all results. Wouldn't recommend it though: could take a while, but you do you.
    if len(cache) >= max_response_cache:
           break

C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\aleja\.conda\envs\PythonData\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [13]:
cache

[{'body': 'Ok eff beating around the bush. I got 1k to sacrifice win or lose.. Hook a brethren up with some non financial advice about what I should do I on Monday.... \n$GME or $AMC I’m down.\n\n\nCause I don’t know wtf I’m doinnggg... kind of',
  'created_utc': 1612058459,
  'created': 1612083659.0},
 {'body': 'I earn less than 1 usd per day! I bought 1 gme from etoro just to support the movement! Hurrah from Indonesia!',
  'created_utc': 1612058458,
  'created': 1612083658.0},
 {'body': "Jesus.\n\nDo you ever walk out your front door, forget it's not there, and faceplant? \n\nPraying you get a decent quote from a contractor bro, I would also reccomend a sturdy railing.\n\nGME to the moon!\n💎🖐🚀",
  'created_utc': 1612058457,
  'created': 1612083657.0},
 {'body': '💎👐 🚀🚀🚀🚀🚀🚀🚀 only ppl to see GME fly by will be 🛸🛸🛸🛸🛸',
  'created_utc': 1612058454,
  'created': 1612083654.0},
 {'body': 'DFW here!! About 5 miles away from the $GME home office. Thank you all for your support in keeping Tex

In [17]:
df = pd.DataFrame(cache)
df['created_utc']=pd.to_datetime(df['created_utc'],unit='s').dt.date
df

,body,created_utc,created
0,Ok eff beating around the bush. I got 1k to sa...,2021-01-31,1.612084e+09
1,I earn less than 1 usd per day! I bought 1 gme...,2021-01-31,1.612084e+09
2,Jesus.\n\nDo you ever walk out your front door...,2021-01-31,1.612084e+09
3,💎👐 🚀🚀🚀🚀🚀🚀🚀 only ppl to see GME fly by will be ...,2021-01-31,1.612084e+09
4,DFW here!! About 5 miles away from the $GME ho...,2021-01-31,1.612084e+09
...,...,...,...
9995,Thank you. What's your feeling about this move...,2021-01-30,1.612067e+09
9996,Is it still worth putting money into GME share...,2021-01-30,1.612067e+09
9997,If the tab is not picked up by the hedge funds...,2021-01-30,1.612067e+09
9998,"This is not a bad theory, BUT are you not miss...",2021-01-30,1.612067e+09
